# **IN4080 - 2023**

# **Mandatory Assignment 2** - Alessia Sanfelici

**Part 1	– Exploring	the NLTK	tagger landscape**

**Exercise	1a:	Data	Split**

For our first experiments, we limit ourselves to the news section of the Brown corpus and split it into a 
training (90%) and a validation (10%) set. (We don’t need a test set for the moment, but will build one in 
part 3.) Moreover, we use the universal tagset instead of the default one.

You should store your data split in the variables news_train and news_val. Note that you may need 
to download the corpus first.

In [82]:
import nltk
nltk.download('universal_tagset')
from nltk.corpus import brown
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score 

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\sanfe\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [57]:
sents = brown.tagged_sents(categories = 'news', tagset = 'universal')

In [58]:
sents

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [59]:
news_train, news_val = train_test_split(sents, test_size = 0.1, random_state = 42)

**Exercise 1b:	Most	Frequent	Class	Baseline**

The distribution of part-of-speech tags is typically quite skewed, with the most frequent class in general 
being common nouns. As a simple baseline, we should thus know how a model that always predicts the 
same (most frequent) class performs. This can be done with nltk.DefaultTagger. Note that we are using the universal tagset, so the 
most frequent tag is not named NN. Evaluate it on the validation set and report the accuracy

In [60]:
tags = [tag for (word, tag) in sents[0]]
max_tag = nltk.FreqDist(tags).max()
print("The most frequent tag is called", max_tag)

The most frequent tag is called NOUN


In [61]:
default_tagger = nltk.DefaultTagger('NOUN')
default_accuracy = default_tagger.accuracy(news_val)
print("Accuracy:")
print(default_accuracy)

Accuracy:
0.2996151189183855


**Exercise	1c:	Naïve	Bayes	Unigram	Tagger**

One of the first models discussed in course is a Naïve Bayes classifier that relies only on the current word 
and does not take any context into account. This model is available as nltk.UnigramTagger. Report the accuracy on the 
validation set. How does the accuracy on the universal tagset differ from the one reported on the default 
tagset in the NLTK book?

In [62]:
unigram_tagger = nltk.UnigramTagger(news_train)
unigram_accuracy = unigram_tagger.accuracy(news_val)
print("Accuracy:")
print(unigram_accuracy)

Accuracy:
0.8853251751702359


The accuracy in this case results to be lower than the one reported on the default tagset in the NLTK book. It is lower of around 5%.

**Exercise	1d:	Bigram	HMM	Tagger**

In the lectures, we spent quite some time on the HMM tagger. Evaluate it on the validation set and report the result.

In [63]:
hmm = nltk.HiddenMarkovModelTagger.train(news_train)
hmm_accuracy = hmm.accuracy(news_val)
print("Accuracy:")
print(hmm_accuracy)

Accuracy:
0.9121681634264285


**Exercise	1e:	Perceptron	with	greedy	decoding**

In the lectures, we have shortly discussed Matthew Honnibal’s proposal of a structured perceptron
tagger with greedy decoding. He argued that an extended set of features is more helpful for tagging than 
exact (Viterbi) decoding. Evaluate it on the validation set and report the result.

In [64]:
perc = nltk.PerceptronTagger(load = False)
perc.train(news_train)
perc_accuracy = perc.accuracy(news_val)
print("Accuracy:")
print(perc_accuracy)

Accuracy:
0.9650646402842199


Summarize the results of the previous exercises and discuss them in a few sentences. Do the accuracies 
correspond to your expectations?

In [65]:
accuracies = [default_accuracy, unigram_accuracy, hmm_accuracy, perc_accuracy]
taggers = ["Default Tagger", "Unigram Tagger", "Hidden Markov Model Tagger", "Perceptron Tagger"]

df_accuracies = pd.DataFrame(accuracies, columns = ["Accuracy"], index = taggers)
df_accuracies

,Accuracy
Default Tagger,0.299615
Unigram Tagger,0.885325
Hidden Markov Model Tagger,0.912168
Perceptron Tagger,0.965065


The obtained results are shown in the previous dataframe. The Defaul Tagger results to be the worst choice in this case, with an accuracy that is very small with respect to the other methods. For Unigram Tagger, HMM tagger and Perceptron tagger, the results ar similar, but the best one is the Percepton tagger. Its accuracy is around 96.5%.

...... discussion ..... what expected?

**Part	2	– Greedy	LR	taggers	and	feature	engineering**

**Exercise	2a:	Getting	started	with	a	greedy	logistic	regression tagger**

In [66]:
class ScikitGreedyTagger(nltk.TaggerI):
    def __init__(self, features, clf=LogisticRegression(max_iter = 400)):
        self.features = features
        self.classifier = clf
        self.vectorizer = DictVectorizer()

    def train(self, train_sents):
        train_feature_sets = []
        train_labels = []
        for tagged_sent in train_sents:
            history = []
            untagged_sent = nltk.tag.untag(tagged_sent)
            for i, (word, tag) in enumerate(tagged_sent):
                feature_set = self.features(untagged_sent, i, history)
                train_feature_sets.append(feature_set)
                train_labels.append(tag)
                history.append(tag)
        x_train = self.vectorizer.fit_transform(train_feature_sets)
        y_train = np.array(train_labels)
        self.classifier.fit(x_train, y_train)

    def tag(self, sentence):
        test_features = []
        history = []
        for i, word in enumerate(sentence):
            featureset = self.features(sentence, i, history)
            test_features.append(featureset)
        X_test = self.vectorizer.transform(test_features)
        tags = self.classifier.predict(X_test)
        return zip(sentence, tags)

In [67]:
def pos_features(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
    return features

In [68]:
lr_tagger = ScikitGreedyTagger(pos_features)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:
0.924010658245337


How does the accuracy of this tagger compare to the taggers tested in part 1?

The accuracy of this tagger results to be good with respect to the results of the previous part: the tagger reaches the second position for highest accuray.

**Exercise	2b:	Adding	word	context	features**

The basic feature function contains the previous and the current word. Also add the next word and the 
word before the previous one. Describe which combination works best and keep it for the next 
experiment.

In [69]:
# Evaluation of previous word, current word and the word before the previous one

def pos_features_1(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
        features["2_prev_word"] = "<START>"
    elif i == 1:
        features["prev_word"] = sentence[i-1]
        features["2_prev_word"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
        features["2_prev_word"] = sentence[i-2]
    return features

lr_tagger = ScikitGreedyTagger(pos_features_1)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:
0.9243067206158098


In [70]:
# Evaluation of previous word, current word and next word

def pos_features_2(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
    try:
        features["next_word"] = sentence[i+1]
    except:
        features["next_word"] = "<END>"
    return features

lr_tagger = ScikitGreedyTagger(pos_features_2)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:
0.934372841211882


In [71]:
# Evaluation of previous word, current word, next word and the word before the previous one

def pos_features_both(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
        features["2_prev_word"] = "<START>"
    elif i == 1:
        features["prev_word"] = sentence[i-1]
        features["2_prev_word"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
        features["2_prev_word"] = sentence[i-2]
    try:
        features["next_word"] = sentence[i+1]
    except:
        features["next_word"] = "<END>"
    return features

lr_tagger = ScikitGreedyTagger(pos_features_both)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:
0.933978091384585


The combination containing the previous word, the current word and the next word works a bit better than the other combinations.

**Exercise	2c:	Adding	transition	features**

Modify the feature function to include the tag predicted at the previous position. Does this help? What 
about a trigram model that includes the two previously predicted tags?

In [107]:
def pos_features_tag(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
        features["prev_tag"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
        features["prev_tag"] = history[i-1]
    return features

lr_tagger = ScikitGreedyTagger(pos_features_tag)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:


IndexError: list index out of range

In [108]:
def pos_features_trigram(sentence, i, history):
    features = {"curr_word": sentence[i]}
    if i == 0:
        features["prev_word"] = "<START>"
        features["prev_tag"] = "<START>"
        features["2_prev_tag"] = "<START>"
    elif i == 1:
        features["prev_word"] = sentence[i-1]
        features["prev_tag"] = history[i-1]
        features["2_prev_tag"] = "<START>"
    else:
        features["prev_word"] = sentence[i-1]
        features["prev_tag"] = history[i-1]
        features["2_prev_tag"] = history[i-2]
    return features

lr_tagger = ScikitGreedyTagger(pos_features_trigram)
lr_tagger.train(news_train)
print("Accuracy:")
print(lr_tagger.accuracy(news_val))

Accuracy:


IndexError: list index out of range

**Exercise	2d:	Even	more	features**

Try to add more features to get an even better tagger. Only the fantasy sets limits to what you may 
consider. Some ideas: Extract suffixes and prefixes from the current, previous or next word. Is the 
current word a number? Is it capitalized? Does it contain capitals? Does it contain a hyphen? etc. What is 
the best feature set you can come up with? Train and test various feature sets and select the best one.
If you use sources for finding tips about good features (like articles, web pages, NLTK code, etc.) make 
references to the sources and explain what you got from them.

**Exercise	2e:	Regularization**

As in the previous assignment, we will study the effect of different regularization strengths now. In scikitlearn, regularization is expressed by the parameter C. A smaller C means stronger regularization. Try with 
C in [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0] and see which value which yields the best result. You can also try 
additional values.

In [109]:
c = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

#create a for cycle

Summarize your experiments to make clear which set of features and parameters provide the best 
results, and what the corresponding accuracy score is. Did you manage to outperform the perceptron 
tagger? If not, where do you think the bottleneck of your current tagger lies?

**Part	3	– Training	and	testing	on	a	larger	corpus**

The Brown corpus covers 15 different genres, but we have only explored the news genre so far. In this 
part, we will retrain the most promising taggers on an extended set of genres and test them on held-out 
data.


**Exercise	3a:	Compile	the	extended	training	and	test	data**

The NLTK book, chapter 2.1.3, lists the names of the 15 genres available in the Brown corpus. We will set 
two genres aside for testing: hobbies and adventure. For training, we will use the news training set 
prepared for the previous exercises, as well as the data from the remaining 12 genres. Prepare the 
corpus as described and store the datasets in the variables all_train, hobbies_test and 
adventure_test. We will not use news_val in this part. Make sure to use the universal tagset.

In [72]:
all_train = brown.tagged_sents(categories = ['news', 'editorial', 'reviews', 'religion', 'lore', 'belles_lettres', 
                                             'government', 'crime', 'science_fiction', 'romance', 'humor', 'learned', 'mystery'], 
                                             tagset = 'universal')
hobbies_test = brown.tagged_sents(categories = 'hobbies', tagset = 'universal')
adventure_test = brown.tagged_sents(categories = 'adventure', tagset = 'universal')

**Exercise	3b:	Evaluate	the	taggers**

Identify the most successful tagger from part 1 and the best setup from part 2. Retrain both of them on 
all_train and evaluate them separately on the two test genres. Report the results and discuss them 
briefly: Which of the two genres is “easier”? How well do the two taggers generalize to unseen genres?


In [73]:
perc = nltk.PerceptronTagger(load = False)
perc.train(all_train)
perc_accuracy_h = perc.accuracy(hobbies_test)
print("Accuracy hobbies:")
print(perc_accuracy_h)
perc_accuracy_a = perc.accuracy(adventure_test)
print("Accuracy adventures:")
print(perc_accuracy_a)

Accuracy hobbies:
0.9676240208877285
Accuracy adventures:
0.9735081191774105


In [ ]:
# Point 2 best???

**Exercise	3c:	Confusion	matrix**

The accuracy gives us a high-level overview of the performance of a tagger, but we may be interested in 
finding out more details about where the tagger makes the mistakes. The universal tagset is reasonably 
small, so we can produce a confusion matrix. Take a look at https://www.nltk.org/api/nltk.tag.api.html
and make a confusion matrix for the results. Pick the results of one test set and one tagger. Make sure 
you understand what the rows and columns are. Which pairs of tags are most easily confounded?
You can find the documentation of the tagset in the following link, but note that NLTK uses an earlier, 
slightly different version of the tagset: https://universaldependencies.org/u/pos/index.html

In [76]:
print(perc.confusion(adventure_test))

     |                             C           N           P           V       |
     |           A     A     A     O     D     O     N     R     P     E       |
     |           D     D     D     N     E     U     U     O     R     R       |
     |     .     J     P     V     J     T     N     M     N     T     B     X |
-----+-------------------------------------------------------------------------+
   . |<10929>    .     .     .     .     .     .     .     .     .     .     . |
 ADJ |     . <3018>    5   120     .     .   156     1     .     1    63     . |
 ADP |     .    12 <6898>   37     3     .    11     .     3    99     6     . |
 ADV |     .   110    55 <3593>    9    15    64     .     .    25     8     . |
CONJ |     .     .     5     . <2153>   12     1     .     .     .     2     . |
 DET |     .     .    20    13     2 <8076>    3     2    39     .     .     . |
NOUN |     .   167     2    38     .     .<13009>   16     1     8   113     . |
 NUM |     .     1     .    

In [ ]:
# Point 2 best???

**Exercise	3d:	Precision,	recall	and	f-measure**

Finding hints on the NLTK web page linked above, calculate the precision, recall and f-measure for each 
tag and display the results in a table.

In [78]:
print(perc.evaluate_per_tag(adventure_test))

 Tag | Prec.  | Recall | F-measure
-----+--------+--------+-----------
   . | 1.0000 | 1.0000 | 1.0000
 ADJ | 0.9039 | 0.8971 | 0.9005
 ADP | 0.9660 | 0.9758 | 0.9709
 ADV | 0.9381 | 0.9263 | 0.9322
CONJ | 0.9935 | 0.9908 | 0.9922
 DET | 0.9860 | 0.9903 | 0.9881
NOUN | 0.9630 | 0.9742 | 0.9685
 NUM | 0.9585 | 0.9914 | 0.9747
PRON | 0.9914 | 0.9787 | 0.9850
 PRT | 0.9405 | 0.9027 | 0.9212
VERB | 0.9834 | 0.9836 | 0.9835
   X | 1.0000 | 0.0263 | 0.0513



Also calculate the macro precision, macro recall and macro f-measure across all tags.

**Exercise	3e:	Error	analysis**

Sometimes, it makes sense to inspect the output of a machine learning model more thoroughly. Find five 
sentences in the test set where at least one token is misclassified and display these sentences in the 
following format, with both the predicted and gold tags.

In [195]:
tagged_words = brown.tagged_words(categories='adventure', tagset='universal')
sentences = brown.sents(categories='adventure')

In [199]:
# Initialize a counter for misclassified sentences
misclassified_count = 0
i = 1

for sentence, tag in zip(sentences, tagged_words):
    predicted = [tag for word, tag in perc.tag(sentence)]
    if any (predicted_tag != gold_tag[1] for predicted_tag, gold_tag in zip(predicted, tagged_words)):
        print("Sentence", i, ":")
        for word, predicted_tag, gold_tag in zip(sentence, predicted, tagged_words):
            print(f"Word: {word}\tPredicted Tag: {predicted_tag}\tGold Tag: {gold_tag[1]}")
        print("\n")
        misclassified_count += 1
        i += 1
    
    # Stop after finding five misclassified sentences
    if misclassified_count == 5:
        break

Sentence 1 :
Word: He	Predicted Tag: PRON	Gold Tag: NOUN
Word: was	Predicted Tag: VERB	Gold Tag: NOUN
Word: well	Predicted Tag: ADV	Gold Tag: VERB
Word: rid	Predicted Tag: ADJ	Gold Tag: PRON
Word: of	Predicted Tag: ADP	Gold Tag: PRON
Word: her	Predicted Tag: PRON	Gold Tag: VERB
Word: .	Predicted Tag: .	Gold Tag: VERB


Sentence 2 :
Word: He	Predicted Tag: PRON	Gold Tag: NOUN
Word: certainly	Predicted Tag: ADV	Gold Tag: NOUN
Word: didn't	Predicted Tag: VERB	Gold Tag: VERB
Word: want	Predicted Tag: VERB	Gold Tag: PRON
Word: a	Predicted Tag: DET	Gold Tag: PRON
Word: wife	Predicted Tag: NOUN	Gold Tag: VERB
Word: who	Predicted Tag: PRON	Gold Tag: VERB
Word: was	Predicted Tag: VERB	Gold Tag: NOUN
Word: fickle	Predicted Tag: NOUN	Gold Tag: NOUN
Word: as	Predicted Tag: ADP	Gold Tag: .
Word: Ann	Predicted Tag: NOUN	Gold Tag: PRON
Word: .	Predicted Tag: .	Gold Tag: VERB


Sentence 3 :
Word: If	Predicted Tag: ADP	Gold Tag: NOUN
Word: he	Predicted Tag: PRON	Gold Tag: NOUN
Word: had	Predicted Tag: 

In [212]:
misclassified_count = 0

# Initialize a list to store misclassified tokens

misclassified_dfs = []

for sentence, gold_tag in zip(sentences, tagged_words):
    predicted = [tag for word, tag in perc.tag(sentence)]
    if any(predicted_tag != gold_tag for predicted_tag, (_, gold_tag) in zip(predicted, tagged_words)):
        misclassified_tokens = []
        for word, predicted_tag, (_, gold_tag) in zip(sentence, predicted, tagged_words):
            #if predicted_tag != gold_tag:
                misclassified_tokens.append([word, predicted_tag, gold_tag])

        # Create a DataFrame from the list of misclassified tokens
        df = pd.DataFrame(misclassified_tokens, columns = ["Token", "Predicted Tag", "Gold Tag"])
        misclassified_dfs.append(df)
        misclassified_count += 1
        misclassified_tokens = []
    # Stop after finding five misclassified sentences
    if misclassified_count == 5:
        break

# Access and print each DataFrame in the list
for i in range(len(misclassified_dfs)):
    print(f"Misclassified Sentence {i+1}:")
    print(misclassified_dfs[i])
    print("\n")

Misclassified Sentence 1:
  Token Predicted Tag Gold Tag
0    He          PRON     NOUN
1   was          VERB     NOUN
2  well           ADV     VERB
3   rid           ADJ     PRON
4    of           ADP     PRON
5   her          PRON     VERB
6     .             .     VERB


Misclassified Sentence 2:
        Token Predicted Tag Gold Tag
0          He          PRON     NOUN
1   certainly           ADV     NOUN
2      didn't          VERB     VERB
3        want          VERB     PRON
4           a           DET     PRON
5        wife          NOUN     VERB
6         who          PRON     VERB
7         was          VERB     NOUN
8      fickle          NOUN     NOUN
9          as           ADP        .
10        Ann          NOUN     PRON
11          .             .     VERB


Misclassified Sentence 3:
      Token Predicted Tag Gold Tag
0        If           ADP     NOUN
1        he          PRON     NOUN
2       had          VERB     VERB
3   married          VERB     PRON
4       her   

Identify the words that are tagged differently. Comment on each of the differences. Would you say that 
the predicted tag is wrong? Or is there a genuine ambiguity such that both answers are defendable? Or is 
even the gold tag wrong?